In [1]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
con = pyodbc.connect('Trusted_Connection=yes', driver = '{SQL Server}',server = 'reportersql', database = 'allocations_2011')
cursor = con.cursor()

In [151]:
age = pd.read_sql_query("""
 select *

from InfoBaseUSdemov1 ib

where ib.[10786] = 1
 """,con) 
#age25_29.head(5)
age.shape
#65+

(608046, 934)

In [152]:
age = age.drop(['300935820'],axis=1)
a_indx = pd.DataFrame(columns=['variable','count'])
for i,j in zip(range(0,934),age.columns):
    a_indx.loc[i,'variable'] = j
    a_indx.loc[i,'count'] = len(age[age[j]=='1'])

print ('count of row,columns',a_indx.shape),(a_indx.head(5))
 
    

count of row,columns (933, 2)


(None,              variable   count
 0             seq_num       0
 1                 zip       0
 2  one_percent_sample  608046
 3                1004   21947
 4                1005   38095)

In [16]:
path_in = 'C:/Users/varunn/Desktop/PAF Std - August 2019 Delivery Docs/New folder/IDandvariablenamelist.csv'
var_name = pd.read_csv(path_in,encoding = 'unicode_escape')

In [153]:
a_indx = a_indx.drop([0,1,2])
a_indx= a_indx.apply(pd.to_numeric,errors='coerce')

In [17]:
var_name[var_name['Taxonomy ID'].isna()]
var_name = var_name.drop([5470])
var_name['Taxonomy ID']= var_name['Taxonomy ID'].astype(int)
var_name = var_name.rename(columns={"Taxonomy ID": "variable"})

In [103]:
#a1824_indx.dtypes

variable    int64
count       int64
dtype: object

In [104]:
#var_name.dtypes

variable                    int32
Segments and Categories    object
dtype: object

In [154]:
a_indx1 = pd.merge(a_indx,var_name)

In [155]:
path_in = 'C:/Users/varunn/Desktop/PAF Std - August 2019 Delivery Docs/New folder/index/a65plus_indx.csv'
a_indx1.to_csv(path_in)

In [91]:
prop = pd.read_sql_query("""
select *

from 
APScore6 ap6 inner join
APScore7 ap7 on ap7.seq_num=ap6.seq_num inner join
Apscore8 ap8 on ap8.seq_num=ap6.seq_num inner join
APScore9 ap9 on ap9.seq_num=ap6.seq_num inner join
InfoBaseUSdemov1 ib on ib.seq_num=ap6.seq_num

where ap6.[300112699]<>1 and  ib.[1825]=1

 """,con) 
#age25_29.head(5)
prop.shape
#prop score 6-9  consumers automotive insurance brand affinity to Progressive.


(121633, 2375)

In [92]:
df = prop.loc[:,~prop.columns.duplicated()]
df.shape

(121633, 2363)

In [93]:
df1 =  df.sample(n=15000,random_state=1)
#df1 = df

In [94]:
df1.shape

(15000, 2363)

In [95]:
a_indx = pd.DataFrame(columns=['variable','count'])
for i,j in zip(range(0,2364),df1.columns):
    a_indx.loc[i,'variable'] = j
    a_indx.loc[i,'count'] = len(df1[df1[j]=='1'])

print ('count of row,columns',a_indx.shape),(a_indx.head(5))
 

count of row,columns (2363, 2)


(None,              variable  count
 0             seq_num      0
 1                 zip      0
 2  one_percent_sample  15000
 3           300026699   4067
 4           300031699    877)

In [99]:
idx_IN_columns = a_indx[a_indx['count']>550].sort_values(by=['count'],ascending=False)
idx_IN_columns.shape
index = list(idx_IN_columns.index)
good_data = df1.iloc[:,index]

In [126]:
#a_indx = a_indx.drop([0,1,2])
#a_indx= a_indx.apply(pd.to_numeric,errors='coerce')

In [62]:
#a_indx1 = pd.merge(a_indx,var_name)
#path_in = 'C:/Users/varunn/Desktop/PAF Std - August 2019 Delivery Docs/New folder/index/prop_indx_a24.csv'
#a_indx1.to_csv(path_in)
 

In [100]:
good_data = good_data.drop(['one_percent_sample'],axis=1)
good_data= good_data.apply(pd.to_numeric,errors='coerce')
good_data = good_data.fillna(0)
[ col  for col, dt in good_data.dtypes.items() if dt == object]

[]

In [101]:
from sklearn.decomposition import PCA
import prince
import visuals as vs
from mca import *

N_eig = 6
mca_ben = MCA(good_data)
mca_ind = MCA(good_data, benzecri=False)
data = {'Eig': pd.Series(mca_ind.L[:N_eig]),
        'Exp_var': mca_ind.expl_var(greenacre=False,N=N_eig),
        'Eig_ben': pd.Series(mca_ben.L[:N_eig]),
        'Exp_var_ben': mca_ben.expl_var(greenacre=False,N=N_eig),
        'Eig_gre': pd.Series(mca_ben.L[:N_eig]),
        'Exp_var_gre': mca_ind.expl_var(greenacre=True,N=N_eig)}

# 'Indicator Matrix', 'Benzecri Correction', 'Greenacre Correction'
columns = ['Eig', 'Exp_var', 'Eig_ben', 'Exp_var_ben', 'Eig_gre', 'Exp_var_gre']
table2 = pd.DataFrame(data=data, columns=columns).fillna(0)
#table2 = pd.DataFrame(data=data, columns=columns)
table2.index += 1
table2.loc['Σ'] = table2.sum()
table2.index.name = 'Factor'

table2

,Eig,Exp_var,Eig_ben,Exp_var_ben,Eig_gre,Exp_var_gre
Factor,,,,,,
1,0.377742,0.073402,0.142389,0.564283,0.142389,0.551596
2,0.209812,0.040770,0.043810,0.173617,0.043810,0.169713
3,0.136270,0.026480,0.018420,0.072996,0.018420,0.071355
4,0.089608,0.017412,0.007926,0.031409,0.007926,0.030703
5,0.075219,0.014616,0.005569,0.022071,0.005569,0.021575
6,0.061728,0.011995,0.003737,0.014809,0.003737,0.014476
Σ,0.950378,0.184676,0.221851,0.879184,0.221851,0.859417


In [102]:
cont = 'contribution'
table3 = pd.DataFrame(columns=good_data.columns, index=pd.MultiIndex
                      .from_product([[cont], range(1, 5)]))

table3.loc[cont,  :] = mca_ben.cont_c(N=4).T * 1000




In [103]:
table3 = table3.loc['contribution',:].T

In [104]:
# identify facotrs that contribut the most to individual variable
maxValueIndexObj = table3.idxmax(axis=1)
 
print("Max values of row are at following columns :")
print(maxValueIndexObj)

Max values of row are at following columns :
1825         1
15041        1
15045        1
15046        1
10533        3
            ..
1235         4
300334999    4
300368799    2
10866        4
1236         4
Length: 1568, dtype: int64


In [106]:
#append the factor variable to column names

table3 =  pd.concat([table3,maxValueIndexObj],axis=1)
table3=table3.rename({0: "factor"},axis='columns')
table3['variable'] = table3.index
#col_factor_selection.to_csv('factor_analysisv2.csv')
table3.head(5)

,1,2,3,4,factor,variable
1825,0.938978,0.006194,0.014124,0.598405,1,1825
15041,0.938978,0.006194,0.014124,0.598405,1,15041
15045,0.938978,0.006194,0.014124,0.598405,1,15045
15046,0.938978,0.006194,0.014124,0.598405,1,15046
10533,0.174300,0.223728,1.057835,0.737487,3,10533


In [41]:
table3.head(5)

,1,2,3,factor
300026699,0.177437,0.125114,0.204968,3
300031699,0.282779,0.248963,0.630281,3
300034699,0.854782,1.306485,1.475046,3
300035699,0.051715,4.224950,2.463514,2
300037699,0.337257,1.658018,0.208558,2


In [107]:
table3['variable']= table3['variable'].astype(int)
a_indx1 = pd.merge(table3,var_name)
path_in = 'C:/Users/varunn/Desktop/PAF Std - August 2019 Delivery Docs/New folder/index/mca_indx_a34_np.csv'
a_indx1.to_csv(path_in)